#IMPORT

In [ ]:
import tweepy
import json
import pandas as pd
from datetime import date, timedelta
import html
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
%%bash
mkdir treetagger
cd treetagger
# Download the tagger package for your system (PC-Linux, Mac OS-X, ARM64, ARMHF, ARM-Android, PPC64le-Linux).
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
tar -xzvf tree-tagger-linux-3.2.4.tar.gz
# Download the tagging scripts into the same directory.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
gunzip tagger-scripts.tar.gz
# Download the installation script install-tagger.sh.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
# Download the parameter files for the languages you want to process.
# list of all files (parameter files) https://cis.lmu.de/~schmid/tools/TreeTagger/#parfiles
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/italian.par.gz
sh install-tagger.sh
cd ..
sudo pip install treetaggerwrapper

bin/
bin/train-tree-tagger
bin/separate-punctuation
bin/tree-tagger
bin/tree-tagger-flush
cmd/
cmd/lookup.perl
COPYRIGHT
doc/
doc/sigdat95.pdf
doc/nemlap94.pdf
FILES
README
Release-Notes

Italian parameter file installed.
Tagging scripts installed.
Path variables modified in tagging scripts.

You might want to add /content/treetagger/cmd and /content/treetagger/bin to the PATH variable so that you do not need to specify the full path to run the tagging scripts.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 KB 888.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for treetaggerwrapper: filename=treetaggerwrapper-2.3-py3-none-any.whl size=40759 sha256=d02ace11d86acedcfb6146a91911e8a666ea995569bf52972cee66aed5e00734
  Stored in directory: /root/.cache/pip/wheels/53/3f/68/dbea5bf2abd18d7d33c8f3f9a

--2023-04-03 15:10:52--  https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
Resolving cis.lmu.de (cis.lmu.de)... 129.187.148.72, 2001:4ca0:4f01::5
Connecting to cis.lmu.de (cis.lmu.de)|129.187.148.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1889240 (1.8M) [application/x-gzip]
Saving to: ‘tree-tagger-linux-3.2.4.tar.gz’

     0K .......... .......... .......... .......... ..........  2%  210K 9s
    50K .......... .......... .......... .......... ..........  5%  422K 6s
   100K .......... .......... .......... .......... ..........  8% 31.2M 4s
   150K .......... .......... .......... .......... .......... 10% 31.1M 3s
   200K .......... .......... .......... .......... .......... 13%  427K 3s
   250K .......... .......... .......... .......... .......... 16% 28.6M 2s
   300K .......... .......... .......... .......... .......... 18% 35.0M 2s
   350K .......... .......... .......... .......... .......... 21% 25.1M 2s
   400K 

In [ ]:
import treetaggerwrapper

/usr/local/lib/python3.9/dist-packages/treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
/usr/local/lib/python3.9/dist-packages/treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
/usr/local/lib/python3.9/dist-packages/treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
/usr/local/lib/python3.9/dist-packages/treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


#VARIABILI

##GIORNO

In [ ]:
OGGI = date.today()
IERI = OGGI - timedelta(days=1)
OGGI = OGGI.strftime('%Y-%m-%d')
IERI = IERI.strftime('%Y-%m-%d')

## PROFILO

In [ ]:
ID_PROFILI = [395218906, 18935802, 420351046, 29416653, 133790890, 14060262, 
              5893702]

# TWEETS RETRIVAL

In [ ]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAB4SbQEAAAAAq2Oa7eu7zbrg2E4B3xLIURWwDBk%3DXxO5cyKNFlrd9LSxYoP1weQmGep38pJoIyH9AG4Pzom0BKqZWX"
client = tweepy.Client(bearer_token)

In [ ]:
def getTweetsAccount(accountId, startPeriod, endPeriod):
  response = client.get_users_tweets(accountId, start_time=startPeriod, 
                                     end_time=endPeriod, max_results=100)
  tweets = []
  try:
    for tweet in response.data:
      if ('RT @' not in tweet.text):
        tweets.append(tweet.text)
  except:
    print("No Tweets")
  return tweets

In [ ]:
def getAllTweetsAccount(accountId, yesterday, today):
  startTime = [IERI+"T06:00:00Z",IERI+"T12:00:00Z",IERI+"T18:00:00Z", 
            OGGI+"T00:00:00Z"]
  endTime = [IERI+"T12:00:00Z",IERI+"T18:00:00Z",IERI+"T23:59:59Z", 
          OGGI+"T06:00:00Z"]
  tweets = []
  for i in range(len(startTime)):
    tweets.extend(getTweetsAccount(accountId, startTime[i], endTime[i]))
  return tweets


In [ ]:
def getAllTweets(accountIds, yesterday, today):
  tweets = []
  for accountId in accountIds:
    tweets.extend(getAllTweetsAccount(accountId, yesterday, today))
  return tweets

In [ ]:
TWEETS = getAllTweets(ID_PROFILI, IERI, OGGI)

#TWEETS CLEANING

In [ ]:
tweets_df = pd.DataFrame (TWEETS, columns = ['text'])

###DROP DUPLICATES

In [ ]:
tweets_df = tweets_df.drop_duplicates('text',keep='first')

### CLEANING HTML

In [ ]:
def toLower(phrase):
  phrase = phrase.lower()
  return phrase

In [ ]:
def removeEmoji(phrase):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', phrase)

In [ ]:
def removeUrls(sentence):
    http_pattern = re.compile(r"http\S+") 
    cleaned_sentence = re.sub(http_pattern,'',sentence).strip()
    www_pattern = re.compile(r"www\S+") 
    cleaned_sentence = re.sub(www_pattern,'',cleaned_sentence)
    return cleaned_sentence

In [ ]:
def removeSpecialCharactersPunctuations(sentence):
    pattern = re.compile("[^a-zA-Z]+") 
    cleaned_text  = re.sub(pattern,' ',sentence).strip()
    return cleaned_text

In [ ]:
def summaryCleaningFunction(text):
  text = toLower(text)
  text = removeEmoji(text)
  text = removeUrls(text)
  text = removeSpecialCharactersPunctuations(text)
  text = text.lower()
  return text

In [ ]:
tweets_df['text']= tweets_df['text'].apply(summaryCleaningFunction)

In [ ]:
tweets_df

,text
0,morto l attore christo jivkov era giovanni nel...
1,andrew rotich kwemoi vince la maratona di mila...
2,il pap di mahsa amini e di un giovane condanna...
3,terremoto napoli due scosse sul vesuvio in mez...
4,il papa celebra la domenica delle palme dopo i...
...,...
463,lavoro il tasso di occupazione cresce ma l ita...
464,pnrr tra il rischio di ritardi e le scadenze d...
465,ucraina la ricostruzione parte dagli aiuti di ...
466,guerra ucraina russia news wagner bakhmut nost...


#TOKENIZATION

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tweets_df['text'] = tweets_df['text'].apply(word_tokenize)

#LEMMATIZATION

In [ ]:
tagger = treetaggerwrapper.TreeTagger(TAGLANG='it', TAGDIR='treetagger/')

def lemmaSentence(token_words):
    lemma_text=[]
    for word in token_words:
      args = treetaggerwrapper.make_tags(tagger.tag_text(word),
                                   allow_extra = True)[0]
      lemmeOption2=args[2].split("|")
      args=lemmeOption2[0]
      lemma_text.append(args)
    return lemma_text
  
tweets_df['text'] = tweets_df['text'].apply(lemmaSentence)

#STOP WORDS REMOVAL

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def rimozione_stop_words(text, stopwords_list):
  return ([word for word in text if word not in stopwords_list])

In [ ]:
englishStopwords = stopwords.words('english')
italianStopwords = stopwords.words('italian')

tweets_df['text'] = tweets_df['text'].apply(lambda x: rimozione_stop_words(x, englishStopwords))
tweets_df['text'] = tweets_df['text'].apply(lambda x: rimozione_stop_words(x, italianStopwords))

#BI-GRAM

In [ ]:
from nltk import word_tokenize 
from nltk.util import ngrams

bigramList = []
for token in tweets_df['text']:
    bigram = list(ngrams(token, 2))
    bigramList.append(bigram)

#TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(use_idf=True, ngram_range=(1,2))
tv_model = tv.fit(tweets_df['text'].astype(str))
tweets_tfidf = tv.transform(tweets_df['text'].astype(str))
feature_names = tv.get_feature_names_out()
dense = tweets_tfidf.todense()
lst1 = dense.tolist()
df = pd.DataFrame(lst1, columns=feature_names)
lista_parole = df.T.sum(axis=1)

In [ ]:
pd.DataFrame(lista_parole.sort_values(ascending=False)[:70])

,0
avere,7.882899
essere,6.521633
morto,5.169684
primo,3.754612
vincere,3.585444
...,...
milano,1.774384
bezzecchi,1.774214
ferito,1.766402
pizzaut,1.745512


#NER

In [ ]:
!pip install https://huggingface.co/bullmount/it_nerIta_trf/resolve/main/it_nerIta_trf-any-py3-none-any.whl
!pip install spacy-transformers
import spacy_transformers
import spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.2/818.2 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 668.8/668.8 KB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
nlp = spacy.load("it_nerIta_trf")

In [ ]:
from collections import Counter

# Inizializza un dizionario vuoto per tenere traccia delle entità
entities = {}

# Analizza ogni tweet e conta le entità
for tweet in tweets_df['text']:
    doc = nlp(tweet)
    for ent in doc.ents:
        if ent.label_ not in entities:
            entities[ent.label_] = Counter()
        entities[ent.label_][ent.text] += 1

# Stampa il ranking delle entità per ogni tipo
for label, entity_counts in entities.items():
    print("Entità più comuni per", label)
    for entity, count in entity_counts.most_common():
        print("\t", entity, count)

ValueError: ignored